Step 1: Create a CSV of tweets from the list of top authors per MBTI type. The
script screens out apparent retweets and non-English speakers. From the
remaining tweets, the script stores 102 tweets (a number divisible by 3) from
each of the 100 users per Myers-Briggs type, for a total of 163,200 tweets.
These tweets are exported to CSV files, one per type, for later analysis.

In [1]:
import re
import sys
LIBPATH = r'D:\Springboard_DataSci\Assignments\Lib'
if LIBPATH not in sys.path:
    sys.path.insert(0, LIBPATH)
import pandas as pd
from langdetect import detect, LangDetectException
import tweepy
import Tweepy_keys
import TimeTracker

USERS_PER_MBT = 100 
TWEETS_PER_USER = 102 #Ideally, divisible by 3
DF_COLUMNS = ['Screen name', 'MBTI', 'Time', 'Tweet']

Get all the keys, tokens, and authorizations.

In [2]:
consumer_key = Tweepy_keys.consumer_key
consumer_secret = Tweepy_keys.consumer_secret
access_token = Tweepy_keys.access_token
access_token_secret = Tweepy_keys.access_token_secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

Get the API.

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=True)
path = r'D:\Springboard_DataSci\Twitter_MBTI_predictor'

Set up the 16 types.

In [4]:
letters = [['E', 'I'], ['S', 'N'], ['F', 'T'], ['J', 'P']]
MB_types = []
for i in range(16):
    MB_types.append(letters[0][i//8%2] + letters[1][i//4%2]
                     + letters[2][i//2%2] + letters[3][i%2])

Proceed with the download. The algorithm screens for tweets that are likely English and throws out authors that are not likely English.

In [5]:
stopwatch = TimeTracker.TimeTracker() # Don't include setup.
for i, MB_type in enumerate(MB_types):
    # Get all the bios and users of the current MB type.
    DF_of_tweets_exists = False
    bios = pd.read_csv(path + '\\Data Input\\' + MB_type + '_bios.csv')
    print('\nCollecting', MB_type, 'tweets:', i+1, 'out of', len(MB_types))
    users = 0
    df_tweets = pd.DataFrame(columns=DF_COLUMNS)
    
    for j, handle in enumerate(bios['Screen name']):
        tweets = []
        # Get the next user of that type.
        bio = str(bios['Bio'].iloc[j]) # str() avoids issues with nan
        non_current_types = list(set(MB_types).difference({MB_type}))
        if any(other_type in bio for other_type in non_current_types):
            print(handle + ' may identify as multiple Myers-Briggs types, not '
                  + 'just ' + MB_type + '. Skipping this user.')
            continue

        count_eng = count_non_eng = 0
        try:
            n_useful_tweets = 0
            for status in tweepy.Cursor(
                    api.user_timeline, screen_name='@' + handle,
                    tweet_mode="extended").items():
                tweet = status.full_text
                # Filter out any of the following:

                # 1. Retweets and tweets with fewer than five words
                if 'http' in tweet: #Trim out any web addresses.
                    tweet = tweet[
                        :[m.start() for m in re.finditer('http', tweet)][0]]
                if tweet[:2] == 'RT' or len(tuple(tweet.split(' '))) < 5:
                    continue

                # 2. Tweets with an undecypherable langauge
                try:
                    lang = detect(tweet)                
                except LangDetectException:
                    continue
                
                # Denote whether the tweet appears to be in English.
                if detect(tweet) == 'en':
                    count_eng += 1
                    n_useful_tweets += 1
                    tweets.append([handle, MB_type, status.created_at,
                                   status.full_text])
                else:
                    count_non_eng += 1
                
                # If most of the tweets are in English, save them.
                if (count_eng+count_non_eng) % (USERS_PER_MBT//3) == 0:
                    likelyEnglishUser = count_eng/(count_eng+count_non_eng)\
                        > 0.75
                    if not likelyEnglishUser:
                        print('\t', handle, 'may be a non-English speaker. '
                              'Skipping to the next one.')
                        break
                if n_useful_tweets >= TWEETS_PER_USER:
                    print("\tChecking user's language...passed."
                          " Saving tweets of", handle)
                    break
            else:
                print('No more tweets for this user. Skipping to next')
                continue
    
        except tweepy.TweepError:
            print("\tDownloading " + handle + "'s tweets failed.",
                  'Proceeding to next one')
        else:
            if likelyEnglishUser:
                users += 1
                print(users, 'out of', USERS_PER_MBT, 'complete for', MB_type)
                df_tweets = df_tweets.append(
                    pd.DataFrame(tweets, columns=DF_COLUMNS),
                    ignore_index=True)
                if users >= USERS_PER_MBT:
                    break
            
    print('Elapsed time for ' + MB_type + ': ' + stopwatch.getElapsedTime())
    df_tweets.to_csv(path + '\\Data Output\\' + MB_type + '_tweets.csv',
                      index=False)


	 boom13003 may be a non-English speaker. Skipping to the next one.
	 mixdukdoi may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of flowerene329
1 out of 100 complete for ESFJ
	Checking user's language...passed. Saving tweets of _CryptoCurator
2 out of 100 complete for ESFJ
	Checking user's language...passed. Saving tweets of Larrykenn
3 out of 100 complete for ESFJ
	 x_432f may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of RichardMBrooks
4 out of 100 complete for ESFJ
	 xr_xb may be a non-English speaker. Skipping to the next one.
	 rmxad may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of jaakotodile
5 out of 100 complete for ESFJ
	 MalazLuay may be a non-English speaker. Skipping to the next one.
	 Mariam_Yagoob may be a non-English speaker. Skipping to the next one.
	 ponceponn may be a non-English speaker. Ski

	 alaamalab may be a non-English speaker. Skipping to the next one.
	 solra_1521 may be a non-English speaker. Skipping to the next one.
	 maariaeduardase may be a non-English speaker. Skipping to the next one.
	 DeeMoselakgomo may be a non-English speaker. Skipping to the next one.
	 fuckngrckwell may be a non-English speaker. Skipping to the next one.
	 LadyHairLoopies may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of ShanettaJamere
35 out of 100 complete for ESFJ
	Checking user's language...passed. Saving tweets of AndeeConstine
36 out of 100 complete for ESFJ
	 Sergiololo12 may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of Joannotations
37 out of 100 complete for ESFJ
	 coeIhuda may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of MacColsonTV
38 out of 100 complete for ESFJ
	 KianoushDaryae2 may be a non-Englis

	 1o_ii77 may be a non-English speaker. Skipping to the next one.
	 swtnpositionsfy may be a non-English speaker. Skipping to the next one.
No more tweets for this user. Skipping to next
	 llxi56_ may be a non-English speaker. Skipping to the next one.
ChrystinaNoel may identify as multiple Myers-Briggs types, not just ESFJ. Skipping this user.
	 mrym_yosf may be a non-English speaker. Skipping to the next one.
	 youronlinebae may be a non-English speaker. Skipping to the next one.
	 _alihariri may be a non-English speaker. Skipping to the next one.
No more tweets for this user. Skipping to next
	 rshaafira may be a non-English speaker. Skipping to the next one.
	 IkoMortensenTan may be a non-English speaker. Skipping to the next one.
	 asp1610 may be a non-English speaker. Skipping to the next one.
AneenaSevla may identify as multiple Myers-Briggs types, not just ESFJ. Skipping this user.
	 marivicentin may be a non-English speaker. Skipping to the next one.
	 anjcdz may be a non-Engl

	Checking user's language...passed. Saving tweets of ayuu_naa
72 out of 100 complete for ESFJ
	 __Gelareh may be a non-English speaker. Skipping to the next one.
	 lllllllk_ may be a non-English speaker. Skipping to the next one.
	 KermitLaChiara may be a non-English speaker. Skipping to the next one.
	 aaadeleen may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of James_Woolley
73 out of 100 complete for ESFJ
	Checking user's language...passed. Saving tweets of taena_kimth
74 out of 100 complete for ESFJ
	Checking user's language...passed. Saving tweets of KrishnaBayuR
75 out of 100 complete for ESFJ
	 adevogabi may be a non-English speaker. Skipping to the next one.
No more tweets for this user. Skipping to next
	 iHassan_HA may be a non-English speaker. Skipping to the next one.
	 Pedrohnrqu may be a non-English speaker. Skipping to the next one.
	 tiramisy0u_ may be a non-English speaker. Skipping to the next one.
	 TB54362414

	 nasuhla may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of MadelineeTate
90 out of 100 complete for ESFJ
	 isaxii9 may be a non-English speaker. Skipping to the next one.
	 rizinddah may be a non-English speaker. Skipping to the next one.
	 Yuri_as9 may be a non-English speaker. Skipping to the next one.
	 alejoina may be a non-English speaker. Skipping to the next one.
	 melissavian may be a non-English speaker. Skipping to the next one.
	 pyrite_Jaehyuk may be a non-English speaker. Skipping to the next one.
	 mutialailani may be a non-English speaker. Skipping to the next one.
	 RawanMirza may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of cndmeilita
91 out of 100 complete for ESFJ
	 GXTA20 may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of _978_
92 out of 100 complete for ESFJ
	Checking user's language...pass

...

	 KevinCarter_ may be a non-English speaker. Skipping to the next one.
	 5xlri may be a non-English speaker. Skipping to the next one.
	 ALasmariAmaL may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of andimJULIE
29 out of 100 complete for INTJ
	 sara233_ may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of Stellablue9
30 out of 100 complete for INTJ
	 H0B1S30K may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of josbannon
31 out of 100 complete for INTJ
	 RodrigoTeijeiro may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of NathanBryce
32 out of 100 complete for INTJ
	Checking user's language...passed. Saving tweets of BennyHeightz
33 out of 100 complete for INTJ
	 malimali may be a non-English speaker. Skipping to the next one.
	 ZoyaKafka may be a non-English spe

	Checking user's language...passed. Saving tweets of DadTigerDad
80 out of 100 complete for INTJ
	Checking user's language...passed. Saving tweets of Sara_Jo_Blevins
81 out of 100 complete for INTJ
	Checking user's language...passed. Saving tweets of MrDre_
82 out of 100 complete for INTJ
	 Went_MIA1 may be a non-English speaker. Skipping to the next one.
	 lllll0o0lllll may be a non-English speaker. Skipping to the next one.
Vincent_Hawke may identify as multiple Myers-Briggs types, not just INTJ. Skipping this user.
	Checking user's language...passed. Saving tweets of DanStlMo
83 out of 100 complete for INTJ
	 AngelRosmend may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of Karen_Cadera
84 out of 100 complete for INTJ
	 KumslutMinnie may be a non-English speaker. Skipping to the next one.
	 mobinazzz may be a non-English speaker. Skipping to the next one.
No more tweets for this user. Skipping to next
	 llii1111iii may be a non

	Checking user's language...passed. Saving tweets of kheici
33 out of 100 complete for INTP
No more tweets for this user. Skipping to next
	 boxtiago may be a non-English speaker. Skipping to the next one.
	 khingkhing15 may be a non-English speaker. Skipping to the next one.
	 W087atim may be a non-English speaker. Skipping to the next one.
	 lollyoongi may be a non-English speaker. Skipping to the next one.
	 byjaelife may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of ericjuta
34 out of 100 complete for INTP
	Checking user's language...passed. Saving tweets of XuanSirius
35 out of 100 complete for INTP
	Checking user's language...passed. Saving tweets of ladyxesphio
36 out of 100 complete for INTP
	 __shaayad_mrll may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of itsLucieLuv
37 out of 100 complete for INTP
	Checking user's language...passed. Saving tweets of eposic
38

	 volumesei may be a non-English speaker. Skipping to the next one.
	 party_poison may be a non-English speaker. Skipping to the next one.
	 yoshicv may be a non-English speaker. Skipping to the next one.
	 mbti_intpp may be a non-English speaker. Skipping to the next one.
	 surenalism may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of Xuan_sirius
68 out of 100 complete for INTP
	Checking user's language...passed. Saving tweets of JahMayCan
69 out of 100 complete for INTP
	 jeliclelim may be a non-English speaker. Skipping to the next one.
	 taxianjwn may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of turtlerocketbks
70 out of 100 complete for INTP
	Checking user's language...passed. Saving tweets of Nothke
71 out of 100 complete for INTP
	 ghostofanti may be a non-English speaker. Skipping to the next one.
robbiemdot may identify as multiple Myers-Briggs types, not just 

	 Hekmatii may be a non-English speaker. Skipping to the next one.
	 senjuwi may be a non-English speaker. Skipping to the next one.
	 mamadrend may be a non-English speaker. Skipping to the next one.
	Checking user's language...passed. Saving tweets of RuthMackenzie76
99 out of 100 complete for INTP
	Checking user's language...passed. Saving tweets of adhdoug
100 out of 100 complete for INTP
Elapsed time for INTP: --- 22.69 hours ---
